## Canny边缘检测
### OpenCV提供了Canny函数来识别边缘。Canny边缘检测算法有5个步骤：使用高斯滤波器对图像进行去噪、计算梯度、在边缘上使用非最大抑制(NMS)、在检测到的边缘上使用双阈值去除假阳性(false positive)，最后还会分析所有的边缘及其之间的连接，以保留真正的边缘并消除不明显的边缘。

In [1]:
import cv2
import numpy as np

img = cv2.imread("flower.jpg")
cv2.imwrite("canny.jpg", cv2.Canny(img, 200, 300))
cv2.imshow("image", img)
cv2.imshow("canny", cv2.imread("canny.jpg"))
cv2.waitKey()
cv2.destroyAllWindows()

## 轮廓检测
### 在计算机视觉中，轮廓检测不仅用来检测图像或者视频帧中物体的轮廓，而且还有其他操作与轮廓检测有关。如：计算多边形边界、形状逼近和计算感兴趣区域。这是与图像数据交互时的简单操作，因为NumPy中的矩形区域可以使用数组切片(slice)来定义。在物体检测（包括人脸）和物体跟踪时会大量使用。

In [2]:
import cv2
import numpy as np

img = np.zeros((200, 200), dtype = np.uint8)   # 创建一个200x200大小的黑色空白图像，
img[50:150, 50:150] = 255                      # 在图像的中央放置一个白色方块

ret, thresh = cv2.threshold(img, 127, 255, 0)  #对图像进行二值化操作
image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # 寻找轮廓
color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # 颜色空间转换
img = cv2.drawContours(color, contours, -1, (0, 255, 0), 2)  # 画出轮廓，-1,表示所有轮廓，画笔颜色为(0, 255, 0)，即Green，粗细为3
cv2.imshow("contours",color)
cv2.waitKey()
cv2.destroyAllWindows()

## 边界框、最小矩形区域和最小闭圆的轮廓
### 可用OpenCV的cv2.findContours函数找到不规则的、歪斜的以及旋转的形状。现实的应用会对目标的边界框、最小矩形面积、最小闭圆特别感兴趣。
### 3.1 绘制矩形
###  函数：cv2.rectangle(img,(380,0),(511,111),(255,0,0),3)，需要确定的就是矩形的两个点（左上角与右下角），颜色，线的类型（不设置就默认）

In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = np.zeros((512,512,3),np.uint8)  #生成一个空彩色图像
cv2.rectangle(img,(20,20),(411,411),(55,255,155),5)
plt.imshow(img,'brg')
plt.show()

<Figure size 640x480 with 1 Axes>

In [4]:
import cv2
import numpy as np

img = cv2.pyrDown(cv2.imread("hammer.jpg", cv2.IMREAD_UNCHANGED))
ret, thresh = cv2.threshold(cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY), 127, 255, cv2.THRESH_BINARY)
image, contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    # find bounding box coordinates
    # 现计算出一个简单的边界框
    x, y, w, h = cv2.boundingRect(c)   # 将轮廓信息转换成(x, y)坐标，并加上矩形的高度和宽度
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)  # 画出矩形

    # find minimum area
    # 计算包围目标的最小矩形区域
    rect = cv2.minAreaRect(c)
    # calculate coordinate of the minimum area rectangle
    box = cv2.boxPoints(rect)
    # normalize coordinates to integers
    box =np.int0(box)
    # 注：OpenCV没有函数能直接从轮廓信息中计算出最小矩形顶点的坐标。所以需要计算出最小矩形区域，
    # 然后计算这个矩形的顶点。由于计算出来的顶点坐标是浮点型，但是所得像素的坐标值是整数（不能获取像素的一部分），
    # 所以需要做一个转换
    # draw contours
    cv2.drawContours(img, [box], 0, (0, 0, 255), 3)  # 画出该矩形

    # calculate center and radius of minimum enclosing circle
    (x, y), radius = cv2.minEnclosingCircle(c)  # 会返回一个二元组，第一个元素为圆心的坐标组成的元组，第二个元素为圆的半径值。
    # cast to integers
    center = (int(x), int(y))
    radius = int(radius)
    # draw the circle
    img = cv2.circle(img, center, radius, (0, 255, 0), 2)

cv2.drawContours(img, contours, -1, (255, 0, 0), 1)
cv2.imshow("contours", img)
cv2.waitKey()
cv2.destroyAllWindows()

error: ..\..\..\modules\imgproc\src\pyramids.cpp:856: error: (-215) !_src.empty() in function cv::pyrDown_


## 4. 凸轮廓与Douglas-Peucker算法
### 可通过OpenCV的cv2.arcLength函数来得到轮廓的周长信息。

In [1]:
epsilon = 0.01 * cv2.arcLength(cnt, True)
approx = cv2.approxPolyDP(cnt, epsilon, True)

NameError: name 'cv2' is not defined

### 可通过OpenCV来有效地计算一个近似多边形，多边形周长与源轮廓周长之比就为ε。
### 为了计算凸形状，需要用OpenCV的cv2.convexHull函数来获取处理过的轮廓信息，代码为：

In [2]:
hull = cv2.convexHull(cnt)

NameError: name 'cv2' is not defined

## 5. 直线和圆检测
### 5.1 绘制直线　
### 函数为：cv2.line（img,Point pt1,Point pt2,color,thickness=1,line_type=8 shift=0）

In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = np.zeros((512,512),np.uint8)#生成一个空灰度图像
cv2.line(img,(0,0),(511,511),255,5)
plt.imshow(img,'gray')
plt.show()

ModuleNotFoundError: No module named 'cv2'

In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = np.zeros((512,512,3),np.uint8)#生成一个空彩色图像
cv2.line(img,(0,0),(511,511),(0,255,0),5)
plt.imshow(img,'brg')
plt.show()

ModuleNotFoundError: No module named 'cv2'

## 5.2 直线检测
### 直线检测可通过HoughLines和HoughLinesP函数来完成，它们仅有的差别是：第一个函数使用标准的Hough变换，第二个函数使用概率Hough变换。HoughLinesP函数之所以成为概率版本的Hough变换是因为它只通过分析点的子集并估计这些点都属于一条直线的概率，这是标准Hough变换的优化版本，该函数的计算代价会少一些，执行会变得更快。

In [5]:
import cv2
import numpy as np

img = cv2.imread("lines.jpg")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 120)
minLineLength = 20
maxLineGap = 5
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength, maxLineGap)

for x1, y1, x2, y2 in lines[0]:
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

cv2.imshow("edges", edges)
cv2.imshow("lines", img)
cv2.waitKey()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'

### 　HoughLines函数会接收一个由Canny边缘检测滤波器处理过的单通道二值图像，不一定需要Canny滤波器，但是一个经过去噪并只有边缘的图像当作Hough变换的输入会很不错。

### HoughLinesP的参数：

### 需要处理的参数；
### 线段的几何表示rho和theta，一般分别取1和np.pi/180；

### 阈值，低于该阈值的直线会被忽略，Hough变换可以理解为投票箱和投票数之间的关系，每一个投票箱代表一个直线，投票数达到阈值的直线会被保留，其他的会被删除。

### minLineLength和maxLineGap　

## 5.3 绘制圆
### 绘制圆形只需要确定圆心与半径，函数： cv2.circle (img,(380,0),63,(255,0,0),3)

In [6]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = np.zeros((512,512,3),np.uint8)#生成一个空彩色图像
cv2.circle(img,(100,100),50,(55,255,155),5)
plt.imshow(img,'brg')
plt.show()

ModuleNotFoundError: No module named 'cv2'

##  5.4 圆检测
### OpenCV的HoughCircles函数可用来检测圆，它与使用HoughLines函数类似。像用来决定删除或保留直线的两个参数minLineLength和maxLineGap一样，HoughCircles有一个圆心间的最小距离和圆的最小及最大半径。

In [7]:
import cv2
import numpy as np

planets = cv2.imread("planet_glow.jpg")
gray_img = cv2.cvtColor(planets, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(gray_img, 5)
cimg = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 120, param1=100, param2 = 30, minRadius = 0,  maxRadius = 0)
circles = np.uint16(np.around(circles))

for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(planets, (i[0], i[1]), i[2],(0, 255, 0),2)
    # draw the center of the circle
    cv2.circle(planets, (i[0], i[1]), 2, (0, 0,255), 3)

cv2.imwrite("planets_circles.jpg",planets)
cv2.imshow("HoughCircles", planets)
cv2.waitKey()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'

## 5.5 绘制椭圆
### 椭圆涉及到长轴短轴，椭圆圆心，旋转角度等

In [8]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = np.zeros((512,512,3),np.uint8)#生成一个空彩色图像
cv2.ellipse(img,(256,256),(150,100),0,0,180,250,-1)
#注意最后一个参数-1，表示对图像进行填充，默认是不填充的，如果去掉，只有椭圆轮廓了
plt.imshow(img,'brg')
plt.show()

ModuleNotFoundError: No module named 'cv2'